# Install required enviroment

In [6]:
!pip install torch==1.5.0
!pip3 install transformers==3.5.0

In [7]:
!git clone  https://github.com/sberbank-ai/ru-gpts

In [ ]:
!mkdir models/

# Load packages and original Dataset

In [ ]:
import pandas as pd
import numpy as np
import torch
import regex as re

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead, GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

Functions to remove emoji from texts

In [ ]:
def deEmojify(text):
    regrex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  
        u"\U0001F300-\U0001F5FF"  
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F1E0-\U0001F1FF"  
        u"\U00002500-\U00002BEF"  
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  
        u"\u3030"
                      "]+", re.UNICODE)

    return regrex_pattern.sub(r'', text)

In [ ]:
def csv_deEmojify(dfs):
  for i in range(dfs.shape[0]):
    for j in range(dfs.shape[1]):
      splited = (str(dfs.iloc[i,j]).split())
      for u in range(len(splited)):
        splited[u] = deEmojify(splited[u])
      res_str = ' '.join(splited)
      dfs.iloc[i,j] = res_str
  return dfs

Save data to the data variable

In [ ]:
def preproc_data(dataset):
  dataset = csv_deEmojify(dataset)
  try:
    dataset = dataset.rename(columns={"Заголовок + конец сообщения": "start_end", "Только заголовок": "start"})
    dataset = dataset[["start_end", "start"]]
  except:
    print("noup")
  
  return dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_excel("/content/drive/MyDrive/course_work/Headings_wo_emoji.xlsx")

### Extract prefixes for geneartion in the near future

In [ ]:
def make_prefs_dataset(dataset):
  prefixes = data["start"].apply(lambda x: re.findall(r"[\w ,]*[!.)]", x)[0])
  bad_places = []
  bad_values = ["Junior)", "Привет, меня зовут Ольга Ищу Java разработчика Senior.", ")", 
                " Олег, cпeциaлиcт пo пoиcку кaндидaтoв кoмпaнии Сбер Пишу, чтобы предложить вакансию Android Developer."]
  for place in range(len(data)):
    value = prefixes[place]
    if value in bad_values:
      bad_places.append(place)
  
  prefixes[18] = "Привет, меня зовут Ольга"
  prefixes[40] = "Добрый день, Павел :) Меня зовут Ольга, я представляю кадровое ИТ агентство."
  prefixes[42] = "Здравствуйте, Павел Я - Олег, cпeциaлиcт пo пoиcку кaндидaтoв кoмпaнии Сбер"
  prefixes[68] = "Компания Сбер"

  prefixes = prefixes.drop_duplicates()
  pd.DataFrame([prefixes.values]).to_csv("GenerationPrefixes.csv", sep=";", index=False) 


### Save CSV-file with prefixes for generation

In [ ]:
def load_prefixes():
  return pd.read_csv("/content/drive/MyDrive/course_work/GenerationPrefixes.csv", sep=";").transpose().rename(columns={0: "message"})

### Make neseccity files (train.txt; valid.txt)

In [1]:
def make_train_and_valid_files(dataset, train_messages=71):
  train_file = open("train.txt", "w")
  valid_file = open("valid.txt", "w")
  
  # first *train_messages* to train
  for sentence in dataset["start"].values[:train_messages]:
    train_file.write(f"{sentence}\n")
  
  # last *86 - train_messages* for validation
  for sentence in dataset["start"].values[train_messages:]:
    valid_file.write(f"{sentence}\n")
  
  train_file.close()
  valid_file.close()

In [ ]:
make_train_and_valid_files(data)

In [ ]:
prefixes = load_prefixes()

# Work with rugpt3small_based_on_gpt2

### Finetuning

In [8]:
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
    --output_dir=small_results \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 10 \
    --block_size 1024 \
    --overwrite_output_dir

### Manually generation

In [2]:
small_gpt_tokenizer = GPT2Tokenizer.from_pretrained("small_results")
small_gpt_model = GPT2LMHeadModel.from_pretrained("small_results")
model.cuda()

Model generate too long sentences for mine task of headings generating. So try to crop everythin after line break

In [ ]:
def generate_by_prefix(model, tokenizer, input_txt, max_len=200):
  inpt = tokenizer.encode(input_txt, return_tensors="pt")
  if torch.cuda.is_available():
    inpt = inpt.cuda()
  out = model.generate(inpt, max_length=max_len, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=1)
  return tokenizer.decode(out[0])

Function to make an excel file with generated headings

In [ ]:
def make_file_with_outputs(model, tokenizer, file_name):
  generated = []
  for pref in prefixes["message"].values:
    generated.append(generate_by_prefix(model, tokenizer, pref))
  generated = list(map(lambda x: x.split("\n")[0], generated))
  pd.DataFrame(generated).to_excel(file_name, index=False)

In [3]:
make_file_with_outputs(small_gpt_model, small_gpt_tokenizer, "small_gpt.xlsx")

# Work with rugpt3medium_based_on_gpt2

### Finetuning

In [4]:
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
    --output_dir=medium_results \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3medium_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 10 \
    --block_size 128 \
    --overwrite_output_dir \
    --per_gpu_eval_batch_size 1

### Manually generation

In [5]:
medium_gpt_tokenizer = GPT2Tokenizer.from_pretrained("medium_results")
medium_gpt_model = GPT2LMHeadModel.from_pretrained("medium_results")
medium_gpt_model.cuda()

In [9]:
make_file_with_outputs(medium_gpt_model, medium_gpt_tokenizer, "medium_gpt.xlsx")

#Worj with Dungeon model (Probably GPT-2 or GPT-3 finetuned on Russian fairy tales)

### I didn't catch how to finetune this model, but decided to generate smth with it just for fun

### P.S. Now, approximately after 1 month since project end, I have some ideas, but it's too late, ahahah. Probably I'll improve it project once

In [ ]:
dungeon_tokenizer = AutoTokenizer.from_pretrained("Mary222/MADE_AI_Dungeon_model_RUS")
dungeon_model = AutoModelWithLMHead.from_pretrained("Mary222/MADE_AI_Dungeon_model_RUS")
dungeon_model.cuda()

In [ ]:
make_file_with_outputs(dungeon_model, dungeon_tokenizer, "unfinetune_dungeon.xlsx")